In [23]:
import psycopg2
import psycopg2.extras
import os
from dotenv import load_dotenv
import datetime
import pandas as pd
import pandas_ta as ta
import numpy as np
import time
from IPython.display import display
import plotly.graph_objects as go
import statistics

load_dotenv()

# Declare global variables
conn = None
cur = None

def get_db_connection():
    global conn, cur  # Declare globals to modify them inside the function

    if conn is None:
        conn = psycopg2.connect(
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            database=os.getenv("DB_NAME"),
        )
    
    if cur is None:
        cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    
    return conn, cur

def close_db_connection():
    global conn, cur  # Access globals to close them properly

    if cur:
        cur.close()
        cur = None  # Reset cur after closing
    
    if conn:
        conn.close()
        conn = None  # Reset conn after closing

def select_by_token_and_interval(cur, symbol, interval):
        select_query = """SELECT * FROM equity_historical_data WHERE symbol = %s AND interval = %s;"""
        cur.execute(select_query, (symbol, interval))
        return cur.fetchall()

def get_token_list(cur):
        select_query = """SELECT * FROM equity_tokens;"""
        cur.execute(select_query)
        return cur.fetchall()

conn, cur = get_db_connection()

In [24]:
def get_data_with_indicators(cur, symbol):
    
    historical_data = select_by_token_and_interval(cur, symbol, 'day')
# Convert historical data to a pandas DataFrame
    
    historical_data = pd.DataFrame(historical_data, columns=['instrument_token', 'symbol', 'interval', 'date', 'open', 'high', 'low', 'close', 'volume'])

    if len(historical_data) < 1000:
        return False

    historical_data = historical_data.sort_values(by='date', ascending=True)
    # Convert columns to float if they are not already
    historical_data['open'] = historical_data['open'].astype(float)
    historical_data['high'] = historical_data['high'].astype(float)
    historical_data['low'] = historical_data['low'].astype(float)
    historical_data['close'] = historical_data['close'].astype(float)

    
    historical_data['sma_20'] = ta.sma(historical_data['close'], length=20)
    historical_data['sma_50'] = ta.sma(historical_data['close'], length=50)
    historical_data['sma_100'] = ta.sma(historical_data['close'], length=100)
    historical_data['sma_200'] = ta.sma(historical_data['close'], length=200)
    historical_data['upper_band'] = historical_data['sma_200'] + ta.atr(historical_data['high'], historical_data['low'], historical_data['close']/2, length=100)
    historical_data['lower_band'] = historical_data['sma_200'] - ta.atr(historical_data['high'], historical_data['low'], historical_data['close']/2, length=100)

    don = ta.donchian(historical_data['high'], historical_data['low'], lower_length=20, upper_length=50, offset=1)

    historical_data['don_high'] = don['DCU_20_50']
    historical_data['don_low'] = don['DCL_20_50']

    historical_data = historical_data.dropna()

    return historical_data

In [25]:
position = None
results = []

tokens = get_token_list(cur)

for token in tokens:
    historical_data = get_data_with_indicators(cur, token[1])
    display(token[1])
    
    if historical_data is not False:
        capital = 25000
        for i in range(1, len(historical_data)):
            if position is None:
                if historical_data['close'].iloc[i] > historical_data['don_high'].iloc[i] and historical_data['close'].iloc[i] > historical_data['sma_50'].iloc[i] > historical_data['sma_100'].iloc[i]> historical_data['sma_200'].iloc[i]:
                    entry_price = historical_data['close'].iloc[i]
                    sl = entry_price - (entry_price * 0.08)
                    qty = capital / entry_price
                    qty = qty = 1 * round(qty / 1)
                    position = {
                        'entry_price': entry_price,
                        'entry_date': historical_data['date'].iloc[i],
                        'stop_loss': sl,
                        'qty': qty
                    }
            
            elif position is not None:
                if historical_data['low'].iloc[i] <= position['stop_loss']:
                    results.append({
                        'entry_date': position['entry_date'],
                        'entry_price': position['entry_price'],
                        'exit_date': historical_data['date'].iloc[i],
                        'exit_price': historical_data['low'].iloc[i],
                        'qty': qty,
                        'pnl': ((historical_data['low'].iloc[i] - position['entry_price']) / position['entry_price'])*100
                    })
                    capital = capital + (historical_data['close'].iloc[i] - position['entry_price']) * position['qty']
                    position = None  # Reset position after closing
                    
                elif (historical_data['close'].iloc[i] < historical_data['sma_50'].iloc[i]):
                    results.append({
                        'entry_date': position['entry_date'],
                        'entry_price': position['entry_price'],
                        'exit_date': historical_data['date'].iloc[i],
                        'exit_price': historical_data['close'].iloc[i],
                        'qty': qty,
                        'pnl': ((historical_data['close'].iloc[i] - position['entry_price']) / position['entry_price'])*100
                    })
                    capital = capital + (historical_data['close'].iloc[i] - position['entry_price']) * position['qty']
                    position = None  # Reset position after closing
                
                elif historical_data['close'].iloc[i] < historical_data['don_low'].iloc[i]:
                    results.append({
                        'entry_date': position['entry_date'],
                        'entry_price': position['entry_price'],
                        'exit_date': historical_data['date'].iloc[i],
                        'exit_price': historical_data['close'].iloc[i],
                        'qty': qty,
                        'pnl': ((historical_data['close'].iloc[i] - position['entry_price']) / position['entry_price'])*100
                    })
                    capital = capital + (historical_data['close'].iloc[i] - position['entry_price']) * position['qty']
                    position = None  # Reset position after closing
            
    if position:
        results.append({
            'entry_date': position['entry_date'],
            'entry_price': position['entry_price'],
            'exit_date': historical_data['date'].iloc[-1],
            'exit_price': historical_data['close'].iloc[-1],
            'qty': qty,
            'pnl': ((historical_data['close'].iloc[-1] - position['entry_price']) / position['entry_price'])*100
        })
        capital = capital +  (historical_data['close'].iloc[i] - position['entry_price']) * position['qty']
        position = None  # Reset position after closing
        
results = pd.DataFrame(results)
results = results.sort_values(by='exit_date', ascending=True)
results['trailing_pnl'] = results['pnl'].cumsum()
results['pnl_points'] = results['exit_price'].astype(float) - results['entry_price'].astype(float)
results['trailing_pnl_points'] = results['pnl_points'].cumsum()
results['pnl_qty_based'] = results['pnl_points'] * results['qty']
results['trailing_pnl_points'] = results['pnl_points'].cumsum()
results['trailing_pnl_points_qty_based'] = results['pnl_qty_based'].cumsum()
display(results.head(30))
display(position)

'360ONE'

'AADHARHFC'

'AARTIIND'

'AAVAS'

'ACE'

...

'WELSPUNLIV'

'ZEEL'

'ZENSARTECH'

,entry_date,entry_price,exit_date,exit_price,qty,pnl,trailing_pnl,pnl_points,trailing_pnl_points,pnl_qty_based,trailing_pnl_points_qty_based
928,2003-10-19 18:30:00+00:00,29.10,2003-10-20 18:30:00+00:00,26.51,859,-8.900344,-8.900344,-2.59,-2.59,-2224.81,-2224.81
992,2003-10-30 18:30:00+00:00,6.00,2003-11-02 18:30:00+00:00,5.45,4167,-9.166667,-18.067010,-0.55,-3.14,-2291.85,-4516.66
1559,2003-11-04 18:30:00+00:00,34.95,2003-11-06 18:30:00+00:00,35.65,715,2.002861,-16.064149,0.70,-2.44,500.50,-4016.16
993,2003-11-05 18:30:00+00:00,6.54,2003-11-13 18:30:00+00:00,5.90,3498,-9.785933,-25.850082,-0.64,-3.08,-2238.72,-6254.88
298,2003-11-16 18:30:00+00:00,26.47,2003-11-18 18:30:00+00:00,24.15,944,-8.764639,-34.614721,-2.32,-5.40,-2190.08,-8444.96
1366,2003-11-11 18:30:00+00:00,9.41,2003-11-20 18:30:00+00:00,8.51,2657,-9.564293,-44.179014,-0.90,-6.30,-2391.30,-10836.26
1426,2003-11-16 18:30:00+00:00,176.15,2003-11-20 18:30:00+00:00,160.15,142,-9.083168,-53.262182,-16.00,-22.30,-2272.00,-13108.26
1212,2003-11-30 18:30:00+00:00,41.00,2003-12-03 18:30:00+00:00,34.02,610,-17.024390,-70.286572,-6.98,-29.28,-4257.80,-17366.06
1703,2003-11-24 18:30:00+00:00,11.68,2003-12-11 18:30:00+00:00,10.69,2140,-8.476027,-78.762600,-0.99,-30.27,-2118.60,-19484.66
231,2003-12-08 18:30:00+00:00,70.60,2003-12-11 18:30:00+00:00,61.55,354,-12.818697,-91.581297,-9.05,-39.32,-3203.70,-22688.36


None

In [26]:
df = results.sort_values(by=['entry_date', 'exit_date'])
overlapping_trades = []
active_trades = []

for i in range(len(df)):
    row = df.iloc[i]
    
    new_active_trades = []
    for trade in active_trades:
        if trade['exit_date'] >= row['entry_date']:
            new_active_trades.append(trade)
        
    active_trades = new_active_trades
    
    active_trades.append(row)
    
    overlapping_trades.append(len(active_trades))

df['overlapping_trades'] = overlapping_trades
results['overlapping_trades'] = df['overlapping_trades']

In [27]:
def calculate_expectancy(win_rate, avg_win_percent, avg_loss_percent):
    # Convert percentages to decimals
    # win_rate /= 100
    win_rate = win_rate / 100
    
    # Loss rate is 1 - win rate
    loss_rate = 1 - win_rate
    
    # Expectancy formula
    expectancy = (win_rate * avg_win_percent) - (loss_rate * abs(avg_loss_percent))
    
    return expectancy

In [28]:
def calculate_troughs_with_duration(df):
    
    troughs = []
    segment = None
    
    local_low = 0
    
    for i in range(len(df)):
        if df['drawdown'].iloc[i] < 0 and df['drawdown'].iloc[i] < local_low and segment is None:
            local_low = df['drawdown'].iloc[i]
            segment = {
                'index': i,
                'start_date': df['exit_date'].iloc[i],
            }
        elif df['drawdown'].iloc[i] < 0 and df['drawdown'].iloc[i] < local_low and segment is not None:
            local_low = df['drawdown'].iloc[i]
        
        elif df['drawdown'].iloc[i] >= 0 and segment is not None:
            segment['end_date'] = df['exit_date'].iloc[i]
            segment['trough'] = local_low
            troughs.append(segment)
            segment = None
            local_low = 0

    return pd.DataFrame(troughs)  # Return troughs as a DataFrame

In [29]:
results['holding_period'] = results['exit_date'] - results['entry_date']
results['streaks'] = (results['pnl'] > 0).astype(int).diff().ne(0).cumsum()
results['streaks'] = results.groupby(results['streaks'])['pnl'].transform(
    lambda x: list(range(1, len(x) + 1)) if x.iloc[0] > 0 else list(range(-1, -len(x) - 1, -1)))

average_winning_holding_period = results[results['pnl'] > 0]['holding_period'].mean()
average_losing_holding_period = results[results['pnl'] < 0]['holding_period'].mean()
accuracy = (results['pnl'] >= 0).mean() * 100
average_profit = (results[results['pnl'] >= 0]['pnl']).mean()
average_loss = (results[results['pnl'] < 0]['pnl']).mean() 
average_profit_points = (results[results['pnl_points'] >= 0]['pnl_points']).mean()
average_loss_points = (results[results['pnl_points'] > 0]['pnl_points']).mean() 
average_rr = (results[results['pnl_qty_based'] > 0]['pnl_qty_based'].mean() / (-results[results['pnl_qty_based'] < 0]['pnl_qty_based'].mean()))
expectancy = calculate_expectancy(accuracy, average_profit, average_loss)
total_gross_profit = results[results['pnl_qty_based'] > 0]['pnl_qty_based'].sum()
total_gross_loss = results[results['pnl_qty_based'] < 0]['pnl_qty_based'].sum()
profit_factor = total_gross_profit / abs(total_gross_loss)
results['net_current_capital'] = results['trailing_pnl_points_qty_based'] + 2500000
results['rolling_max'] = results['trailing_pnl_points_qty_based'].cummax()
results['drawdown'] = ((results['trailing_pnl_points_qty_based'] - results['rolling_max']))
results['average_drawdown'] = ta.sma(results['drawdown'], length=100)
results['ptc_drawdown'] = (results['drawdown'] / results['net_current_capital']) * 100
troughs_df = calculate_troughs_with_duration(results)

initial_strategy_value = results['trailing_pnl_points_qty_based'].iloc[0] + 2500000
final_strategy_value = results['trailing_pnl_points_qty_based'].iloc[-1] + 2500000
total_strategy_returns = (final_strategy_value - initial_strategy_value) / initial_strategy_value * 100
cagr_strategy_returns = ((final_strategy_value / initial_strategy_value) ** (1 / 21) - 1)*100

display(results)
display(troughs_df)
print("Average winning holding period:", average_winning_holding_period)
print("Average losing holding period:", average_losing_holding_period)
print("Accuracy:", accuracy)
print('Average holding period:', results['holding_period'].mean())
print("Average profit:", average_profit)
print("Average loss:", average_loss)
print("Risk Reward Ratio:", average_rr)
print('Average absolute loss', results[results['pnl_qty_based'] < 0]['pnl_qty_based'].mean())
print('Average absolute profit', results[results['pnl_qty_based'] > 0]['pnl_qty_based'].mean())
print('Maximum absolute loss', results['pnl_qty_based'].min())
print('Maximum absolute profit', results['pnl_qty_based'].max())
print('Average pnl per trade', results['pnl'].mean())
print("Expectancy:", expectancy)
print("Profit factor:", profit_factor)
print("Total gross profit:", total_gross_profit)
print("Total gross loss:", total_gross_loss)
print("Average Drawdown:", results['drawdown'].mean())
print("Drawdow Descriptive Statistics:", results['ptc_drawdown'].describe(percentiles=[0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9]))
print("Return to ADD:", results['pnl'].sum()/-results['drawdown'].mean())
print("Return to MDD:", results['pnl'].sum()/-results['drawdown'].min())
print("Total strategy returns:", total_strategy_returns)
print("Strategy CAGR:", cagr_strategy_returns)
print("Longest losing streak:", results['streaks'].min())
print("Longest winning streak:", results['streaks'].max())
print("Average losing streak:", results[results['streaks'] < 0]['streaks'].mean())
print("Average winning streak:", results[results['streaks'] > 0]['streaks'].mean())
print('Max open position:', results['overlapping_trades'].max())
print('Mean open position:', results['overlapping_trades'].mean())



,entry_date,entry_price,exit_date,exit_price,qty,pnl,trailing_pnl,pnl_points,trailing_pnl_points,pnl_qty_based,trailing_pnl_points_qty_based,overlapping_trades,holding_period,streaks,net_current_capital,rolling_max,drawdown,average_drawdown,ptc_drawdown
928,2003-10-19 18:30:00+00:00,29.10,2003-10-20 18:30:00+00:00,26.51,859,-8.900344,-8.900344,-2.59,-2.59,-2224.81,-2224.81,1,1 days,-1,2497775.19,-2224.81,0.00,NaN,0.000000
992,2003-10-30 18:30:00+00:00,6.00,2003-11-02 18:30:00+00:00,5.45,4167,-9.166667,-18.067010,-0.55,-3.14,-2291.85,-4516.66,3,3 days,-2,2495483.34,-2224.81,-2291.85,NaN,-0.091840
1559,2003-11-04 18:30:00+00:00,34.95,2003-11-06 18:30:00+00:00,35.65,715,2.002861,-16.064149,0.70,-2.44,500.50,-4016.16,3,2 days,1,2495983.84,-2224.81,-1791.35,NaN,-0.071769
993,2003-11-05 18:30:00+00:00,6.54,2003-11-13 18:30:00+00:00,5.90,3498,-9.785933,-25.850082,-0.64,-3.08,-2238.72,-6254.88,5,8 days,-1,2493745.12,-2224.81,-4030.07,NaN,-0.161607
298,2003-11-16 18:30:00+00:00,26.47,2003-11-18 18:30:00+00:00,24.15,944,-8.764639,-34.614721,-2.32,-5.40,-2190.08,-8444.96,6,2 days,-2,2491555.04,-2224.81,-6220.15,NaN,-0.249649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,2024-11-24 18:30:00+00:00,532.05,2025-01-02 18:30:00+00:00,611.75,159,14.979795,11664.496861,79.70,18652.54,12672.30,7316046.78,6,39 days,4,9816046.78,7316046.78,0.00,-57590.4223,0.000000
1073,2024-12-03 18:30:00+00:00,1240.40,2025-01-02 18:30:00+00:00,1223.80,25,-1.338278,11663.158583,-16.60,18635.94,-415.00,7315631.78,13,30 days,-1,9815631.78,7316046.78,-415.00,-57402.1223,-0.004228
12,2024-12-02 18:30:00+00:00,1156.70,2025-01-02 18:30:00+00:00,1297.15,16,12.142301,11675.300884,140.45,18776.39,2247.20,7317878.98,11,31 days,1,9817878.98,7317878.98,0.00,-57162.4223,0.000000
220,2025-01-01 18:30:00+00:00,10519.85,2025-01-02 18:30:00+00:00,10950.70,23,4.095591,11679.396475,430.85,19207.24,9909.55,7327788.53,12,1 days,2,9827788.53,7327788.53,0.00,-56487.0873,0.000000


,index,start_date,end_date,trough
0,1,2003-11-02 18:30:00+00:00,2004-01-21 18:30:00+00:00,-30110.15
1,40,2004-04-27 18:30:00+00:00,2005-01-19 18:30:00+00:00,-42098.66
2,74,2005-01-26 18:30:00+00:00,2005-01-26 18:30:00+00:00,-531.30
3,77,2005-02-06 18:30:00+00:00,2005-02-21 18:30:00+00:00,-14968.05
4,83,2005-02-23 18:30:00+00:00,2005-03-10 18:30:00+00:00,-11288.54
...,...,...,...,...
95,1602,2024-03-12 18:30:00+00:00,2024-04-09 18:30:00+00:00,-8464.45
96,1607,2024-04-09 18:30:00+00:00,2024-10-21 18:30:00+00:00,-176338.50
97,1708,2024-10-21 18:30:00+00:00,2024-11-11 18:30:00+00:00,-20527.25
98,1713,2024-11-11 18:30:00+00:00,2025-01-02 18:30:00+00:00,-20947.64


Average winning holding period: 93 days 13:47:22.700156986
Average losing holding period: 20 days 00:10:30.656934306
Accuracy: 36.829971181556196
Average holding period: 47 days 00:29:02.939481268
Average profit: 33.27855499989134
Average loss: -8.745985556640903
Risk Reward Ratio: 3.8227543537647763
Average absolute loss -5472.196624087591
Average absolute profit 20918.86346938775
Maximum absolute loss -248839.19999999995
Maximum absolute profit 1383664.9
Average pnl per trade 6.731640619511319
Expectancy: 6.73164061951132
Profit factor: 2.2218015723979585
Total gross profit: 13325316.029999997
Total gross loss: -5997527.5
Average Drawdown: -90314.9594005763
Drawdow Descriptive Statistics: count    1735.000000
mean       -2.083487
std         2.497066
min        -8.977058
1%         -8.451681
5%         -7.690930
10%        -6.575994
25%        -3.394398
50%        -0.861815
75%        -0.189658
90%         0.000000
max         0.000000
Name: ptc_drawdown, dtype: float64
Return to ADD

In [30]:
from matplotlib.pyplot import title


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=results['exit_date'],
    y=results['trailing_pnl'],
    mode='lines',
    name='Trailing PnL',
    line=dict(color='blue')
))
fig.show()


fig2 = go.Figure()
fig2.add_trace(go.Scatter(
    x=results['exit_date'],
    y=results['trailing_pnl_points_qty_based'],
    mode='lines',
    name='Trailing PnL Points',
    line=dict(color='blue')
))
fig2.show()


fig3 = go.Figure()
fig3.add_trace(go.Scatter(
    x=results['exit_date'],
    y=results['drawdown'],
    mode='lines',
    name='Drawdowns',
    line=dict(color='blue')
))
fig3.add_trace(go.Scatter(
    x=results['exit_date'],
    y=results['average_drawdown'],
    mode='lines',
    name='Average Drawdowns',
    line=dict(color='green')
))
fig3.add_trace(go.Scatter(
    x=results['exit_date'],
    y=results['drawdown'].cummin(),
    mode='lines',
    name='Highest Drawdowns',
    line=dict(color='red')
))
fig3.show()



fig5 = go.Figure()
fig5.add_trace(go.Scatter(
    x=results['exit_date'],  # Ensure 'exit_date' exists in results
    y=results['pnl_qty_based'],        # Ensure 'pnl' exists in results
    mode='lines',
    name='Qty Based PnL',
    line=dict(color='green')
))
fig5.show()

fig6 = go.Figure()
fig6.add_trace(go.Scatter(
    x=results['exit_date'],  # Ensure 'exit_date' exists in results
    y=results['streaks'],        # Ensure 'pnl' exists in results
    mode='lines',
    name='Streak',
    line=dict(color='green')
))
fig6.show()

fig7 = go.Figure()
fig7.add_trace(go.Scatter(
    x=results['exit_date'],  # Ensure 'exit_date' exists in results
    y=results['qty'],        # Ensure 'pnl' exists in results
    mode='lines',
    name='Qty',
    line=dict(color='green')
))
fig7.show()


fig8 = go.Figure()
fig8.add_trace(go.Scatter(
    x=results['exit_date'],  # Ensure 'exit_date' exists in results
    y=results['ptc_drawdown'],        # Ensure 'pnl' exists in results
    mode='lines',
    name='% wise Drawdown',
    line=dict(color='green')
))
fig8.show()

